In [1]:
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import numpy as np
import json
import math
import csv
import pandas as pd
from scipy import optimize
import torch.optim.lr_scheduler as lr_scheduler

In [ ]:
#define architecture for three waveforms
class Net(nn.Module):                        # model class      
    def __init__(self):                      
        super().__init__()
        self.fc1 = nn.Linear(7, 16)    # four-layered network; number of neurons in each layer
        self.fc2 = nn.Linear(16, 256)   # equals number of output; no. of bias = no. of outputs; weights = no. of(input*output)
        self.fc3 = nn.Linear(256, 16)
        self.fc4 = nn.Linear(16,1)
        self.dropout1 = nn.Dropout(0.1)

    def forward(self, x):              # x is the input vector #
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.tanh(self.fc3(x))
        x = self.fc4(x)                # real value
        return x

In [2]:
data = pd.read_excel('./DataSplit/'+Material+'/_Sine/' + Material +'_Sine'+ '.xlsx')

DataF1 = data['Freq (kHz)'].to_numpy()
DataB1 = data['Bpkpk (mT)'].to_numpy()
DataT1 = data['T'].to_numpy()
DataBmin1 = data['minBpk (mT)'].to_numpy()
DataBmax1 = data['maxBpk (mT)'].to_numpy()
DataBRMS1 = data['RMSB (mT)'].to_numpy()
DataTime1 = data['Period (us)'].to_numpy()
DataIndSine = data['DataIndex'].to_numpy()

DataF = np.log10(DataF1)
DataB = np.log10(DataB1)
DataT = np.log10(DataT1)
DataBmin = np.log10(DataBmin1)
DataBmax = np.log10(DataBmax1)
DataBRMS = np.log10(DataBRMS1)
DataTime = np.log10(DataTime1)

DataF = DataF.reshape((-1,1))
DataB = DataB.reshape((-1,1))
DataT = DataT.reshape((-1,1))
DataBmin = DataBmin.reshape((-1,1))
DataBmax = DataBmax.reshape((-1,1))
DataBRMS = DataBRMS.reshape((-1,1))
DataTime = DataTime.reshape((-1,1))

input_data = np.concatenate((DataF,DataB,DataT,DataBmin,DataBmax,DataBRMS,DataTime),axis=1)
input_tensors = torch.from_numpy(input_data).view(-1,7)
dataset_sine = torch.utils.data.TensorDataset(input_tensors)

C:\Users\himan\AppData\Local\Temp\ipykernel_33336\3732659971.py:12: RuntimeWarning: invalid value encountered in log10
  DataF = np.log10(DataF1)
C:\Users\himan\AppData\Local\Temp\ipykernel_33336\3732659971.py:18: RuntimeWarning: invalid value encountered in log10
  DataTime = np.log10(DataTime1)


In [ ]:
test_loader_sine = torch.utils.data.DataLoader(dataset_sine, batch_size = 128, shuffle = False)

ckpt_sine = torch.load('./Trained_Models/'+Material+'_sine_train.pt')
net_sine = ckpt_sine['model'] # load the trained model
y_pred = []
net_sine.eval()
with torch.no_grad():
    for inputs, in test_loader_sine:
        test_out = net_sine(inputs)
        y_pred.append(test_out)

y_pred = torch.cat(y_pred, dim=0)
sine_test_out = (10**(y_pred.numpy()))*1e3
sine_index    = DataIndSine